In [1]:
import numpy as np
from scipy import stats
import unittest
%run probability_fns.ipynb
%run likelihood_fns.ipynb

...../var/folders/z3/65sxpv5n3csdhfzm0ysx30940000gp/T/ipykernel_61957/3392994918.py:117: RuntimeWarning: divide by zero encountered in log
  LP += np.log(sigma_prior(sigma, lower_0, upper_0, printing, debug))
/var/folders/z3/65sxpv5n3csdhfzm0ysx30940000gp/T/ipykernel_61957/3392994918.py:390: RuntimeWarning: divide by zero encountered in log
  real_LP = np.sum(np.log(probs))
..E
ERROR: test_theta_prior (__main__.ProbabilityFunctionsTestSuite)
Test theta_prior
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/var/folders/z3/65sxpv5n3csdhfzm0ysx30940000gp/T/ipykernel_61957/3392994918.py", line 410, in test_theta_prior
    returned_p = theta_prior(*testValues, printing=True, debug=True)
  File "/var/folders/z3/65sxpv5n3csdhfzm0ysx30940000gp/T/ipykernel_61957/3392994918.py", line 222, in theta_prior
    print_fn(["theta", theta, "mu_0", mu_0, sigma_0, "sigma_0"])
NameError: name 'print_fn' is not defined

---------------------

in theta prior
in theta prior
end is lower than start; can happen and will return 0 which will cause neg infs later: [-inf, 0.0, -1.0, 1.0, inf]
in theta prior

 theta has value 1.5

 mu_0 has value 1.5

 1.0 has value sigma_0
in theta prior

 theta has value 2.5

 mu_0 has value 2.5

 1.0 has value sigma_0

 sigma_0 has value 1.0

 ks plus shift has value [1.5, 2.5]

 thetas has value [-inf, 1.5, 2.5, inf]

 thetaPriors has value [0.3989422804014327, 0.3989422804014327]

 theta log prior internal has value -1.8378770664093453

 thetas has value [-inf, 1.5, 2.5, inf]

 shift has value 0.5

 sigma_0 has value 1.0

 LP has value -1.8378770664093453



----------------------------------------------------------------------
Ran 6 tests in 0.012s

OK


## Likelihood functions and checks
This notebook is to be used as a submodule that contains wrappers for all the likelihood functions used by the ordinal probit model for survey data, and Metropolis-Hastings sampler. There is also an optional testing suite.

In [2]:
def initialize_suite(TestCase):
    loader = unittest.TestLoader()
    suite = loader.loadTestsFromTestCase(TestCase)
    return suite 

In [12]:
def mu_accept(mu, mu_star, guessVector, sigma, thetas, mu_0, sigma_0, sigma_prop,
             printing=False, debug=False):
    """Returns the acceptance probability of a proposal for mu.
    
    Inputs
    -------------------
    mu: current latent mean; scalar value in Reals;
    mu_star: proposal for new latent mean; scalar value in Reals;
    guessVector: guesses / y's. Each should be an integer from 1:k; could be list or vector of ints;
    sigma: latent sd; scalar value in (0, inf);
    thetas: values of latent thresholds defining probit probabilities; scalars in [1.5, k-0.5];
            endpoints are -inf, inf. Length is k+2, where k is the number of possible ordinal choices.
    mu_0: prior mean; scalar value in Reals;
    sigma_0: prior standard devation; scalar value in (0, inf);
    printing: Bool; whether to print messages;
    debug: Bool; whether to run internal error checks.
    
    Outputs
    -------------------
    AP: acceptance probability (ratio); scalar in (0, inf).
    """
    
    A1 = joint_log_probability_mu(guessVector, mu_star, sigma, thetas, mu_0, sigma_0, 
                        printing, debug)
    A2 = joint_log_probability_mu(guessVector, mu, sigma, thetas, mu_0, sigma_0, 
                        printing, debug)
    
    A3 = mu_log_jump_probs(mu, mu_star, sigma_prop, printing, debug)
    
    A = A1 - A2 + A3
    return min(1, np.exp(A))

def sigma_accept(sigma, sigma_star, guessVector, mu, thetas, sigma_prop, lower_0, upper_0,
             printing=False, debug=False):
    """Returns the acceptance probability of a proposal for sigma.
    
    Inputs
    -------------------
    sigma: current latent mean; scalar value in (0, inf);
    sigma_star: proposal for new latent mean; scalar value in (0, inf); 
    guessVector: guesses / y's. Each should be an integer from 1:k; could be list or vector of ints;
    mu: latent mean; scalar value in Reals;
    thetas: values of latent thresholds defining probit probabilities; scalars in [1.5, k-0.5];
            endpoints are -inf, inf. Length is k+2, where k is the number of possible ordinal choices.
    mu_0: prior mean; scalar value in Reals;
    sigma_0: prior standard devation; scalar value in (0, inf);
    printing: Bool; whether to print messages;
    debug: Bool; whether to run internal error checks.
    
    Outputs
    -------------------
    AP: acceptance probability (ratio); scalar in (0, inf).
    """
    A1 = joint_log_probability_sigma(guessVector, mu, sigma_star, thetas, lower_0, upper_0,
                        printing, debug)
    A2 = joint_log_probability_sigma(guessVector, mu, sigma, thetas, lower_0, upper_0,
                        printing, debug)
    
    A3 = sigma_log_jump_probs(sigma, sigma_star, sigma_prop, lower_0, upper_0, printing, debug)
    
    A = A1 - A2 + A3
    return min(1, np.exp(A))

def theta_accept(theta, theta_star, thetas, guessMatrix, mus, sigmas, center, shift, sigma_0, sigma_prop, 
                 lower_0, upper_0, printing=False, debug=False):
    """Returns the acceptance probability of a proposal for theta. A bit inefficient. Note, have to monitor
    here for -infs, as sometimes the proposals reverse order sloppily. 
    
    Inputs
    -------------------
    theta: a current threshold; scalar value in (0, inf);
    theta_star: proposal for new latent mean; scalar value in (0, inf); 
    guessVector: guesses / y's. Each should be an integer from 1:k; could be list or vector of ints;
    mu: latent mean; scalar value in Reals;
    thetas: values of latent thresholds defining probit probabilities; scalars in [1.5, k-0.5];
            endpoints are -inf, inf. Length is k+2, where k is the number of possible ordinal choices.
    mu_0: prior mean; scalar value in Reals;
    sigma_0: prior standard devation; scalar value in (0, inf);
    printing: Bool; whether to print messages;
    debug: Bool; whether to run internal error checks.

    Outputs
    -------------------
    AP: acceptance probability (ratio); scalar in (0, inf).
    """
    
    thetasTemp = thetas.copy()
    thetasTemp[center] = theta_star
#     print("theta", theta, "theta_star",  theta_star, "sigma_prop",sigma_prop, "lower_0", lower_0,
#           "upper_0", upper_0)
    A1 = joint_log_probability_thetas(guessMatrix, mus, sigmas, thetasTemp, shift, sigma_prop,
                                     printing, debug)
    
    A2 = joint_log_probability_thetas(guessMatrix, mus, sigmas, thetas, shift, sigma_prop,
                                     printing, debug)
    
    A3 = theta_log_jump_probs(theta, theta_star, sigma_prop, lower_0, upper_0, printing, debug)

    A = A1 - A2 + A3
    
    if debug:
#         assert -np.inf <= A <= 0.0, "A not in range: A1 {} A2 {} A3 {}".format(A1, A2, A3)
        if np.isneginf(A):
            print('A is -inf: A {}; A1 {} A2 {} A3 {}'.format(A, A1, A2, A3))
    if printing: 
        print("A1 {} A2 {} A3 {}".format(A1, A2, A3))
    if np.isneginf(A):
        return 0.0 # do not move if that would swap a threshold
    else:
        return min(1, np.exp(A))
    
class LikelihoodFunctionsTestSuite(unittest.TestCase):
    def gauss(self, val, mu=0, s=1):
            return np.exp(- (1/2)*
                          ((val-mu)/s)**2) / np.sqrt(2*np.pi*(s**2))
    
    def gauss_trunc(self, x, mu, sigma, lower, upper):
        zeta = (x-mu)/sigma
        alpha = (lower-mu)/sigma
        beta = (upper-mu)/sigma
        Z = stats.norm.cdf(beta) - stats.norm.cdf(alpha)
        return (self.gauss(zeta)) / (sigma*Z)
    
    def test_mu_accept(self):
        #             mu. mu_star  y.    sigma         thetas             mu_0 s_0  s_j
        testValues = [0.0, 1.0, [2, 1], 2.0, [-np.inf, 1.5, 2.5, np.inf], 1.0, 1.0, 2.0]
        mu, mu_star, guessVector, sigma, thetas, mu_0, s_0, s_j = testValues.copy()
        
        returned_A = mu_accept(*testValues, printing=True, debug=True)
        
        real_A1 = np.sum(np.log([max(0, stats.norm.cdf(thetas[guessVector[0]+1], mu_star, sigma) - stats.norm.cdf(thetas[guessVector[0]], mu_star, sigma)), 
                                max(0, stats.norm.cdf(thetas[guessVector[1]+1], mu_star, sigma) - stats.norm.cdf(thetas[guessVector[1]], mu_star, sigma)),
                                self.gauss(mu_star, mu_0, s_0)]))
        real_A2 = np.sum(np.log([max(0, stats.norm.cdf(thetas[guessVector[0]+1], mu, sigma) - stats.norm.cdf(thetas[guessVector[0]], mu, sigma)), 
                                max(0, stats.norm.cdf(thetas[guessVector[1]+1], mu, sigma) - stats.norm.cdf(thetas[guessVector[1]], mu, sigma)),
                                self.gauss(mu, mu_0, s_0)]))
        
        real_A3 = 0
        comb = np.min([1, np.exp(real_A1 - real_A2 + real_A3)])
        self.assertAlmostEqual(returned_A, comb)
        
        
    def test_sigma_accept(self):
        #             s.  s_star  y.    mu         thetas                s_j l1 l2
        testValues = [1.0, 2.0, [2, 1], 2.0, [-np.inf, 1.5, 2.5, np.inf], 2.0, 0, 3]
        sigma, sigma_star, guessVector, mu, thetas, s_j, l1, l2 = testValues.copy()
        returned_A = sigma_accept(*testValues, printing=True, debug=True)
        real_A1 = np.sum(np.log([max(0, stats.norm.cdf(thetas[guessVector[0]+1], mu, sigma_star) - stats.norm.cdf(thetas[guessVector[0]], mu, sigma_star)), 
                                max(0, stats.norm.cdf(thetas[guessVector[1]+1], mu, sigma_star) - stats.norm.cdf(thetas[guessVector[1]], mu, sigma_star)),
                                1/(l2-l1)]))
        real_A2 = np.sum(np.log([max(0, stats.norm.cdf(thetas[guessVector[0]+1], mu, sigma) - stats.norm.cdf(thetas[guessVector[0]], mu, sigma)), 
                                max(0, stats.norm.cdf(thetas[guessVector[1]+1], mu, sigma) - stats.norm.cdf(thetas[guessVector[1]], mu, sigma)),
                                1/(l2-l1)]))
        real_A3 = np.log(self.gauss_trunc(sigma, sigma_star, s_j, l1, l2)) - np.log(self.gauss_trunc(sigma_star, sigma, s_j, l1, l2)) # because symmetric ---> Metropolis
        comb = np.min([1, np.exp(real_A1 - real_A2 + real_A3)])
        self.assertAlmostEqual(returned_A, comb)
        
    def test_theta_accept(self):
        # in this case proposal is in order
        testValues1 = [2.5, 2.6, [-np.inf, 1.5, 2.5, 3.5, np.inf], [[2, 1], [2, 3]], [1.0, 2.0], [1.0, 1.0], 2, 0.5, 1.0, 1.0, 0, 3]
        theta, theta_star, thetas, guessMatrix, mus, sigmas, center, shift, sigma_0, sigma_prop, lower_0, upper_0 = testValues1.copy()
        returned_A = theta_accept(*testValues1, printing=True, debug=True)
        thetasStar = thetas.copy()
        thetasStar[center] = theta_star
        
        real_LP_1 = np.sum(np.log([max(0, 
                                       stats.norm.cdf(thetasStar[guessMatrix[0][0]], mus[0], sigmas[0]) - stats.norm.cdf(thetasStar[guessMatrix[0][0]-1], mus[0], sigmas[0])), 
                                max(0, 
                                    stats.norm.cdf(thetasStar[guessMatrix[0][1]], mus[0], sigmas[0]) - stats.norm.cdf(thetasStar[guessMatrix[0][1]-1], mus[0], sigmas[0])),
                                   max(0, 
                                       stats.norm.cdf(thetasStar[guessMatrix[1][0]], mus[1], sigmas[1]) - stats.norm.cdf(thetasStar[guessMatrix[1][0]-1], mus[1], sigmas[1])), 
                                max(0, 
                                    stats.norm.cdf(thetasStar[guessMatrix[1][1]], mus[1], sigmas[1]) - stats.norm.cdf(thetasStar[guessMatrix[1][1]-1], mus[1], sigmas[1]))]))
        
        
        tempProbsStar = [self.gauss(x, i+1.5, sigma_0) for (i, x) in enumerate(thetasStar[1:-1])]
        real_LP_2 = np.sum(np.log(tempProbsStar))

        
        real_LP_3 = np.sum(np.log([max(0, 
                                       stats.norm.cdf(thetas[guessMatrix[0][0]], mus[0], sigmas[0]) - stats.norm.cdf(thetas[guessMatrix[0][0]-1], mus[0], sigmas[0])), 
                                max(0, 
                                    stats.norm.cdf(thetas[guessMatrix[0][1]], mus[0], sigmas[0]) - stats.norm.cdf(thetas[guessMatrix[0][1]-1], mus[0], sigmas[0])),
                                   max(0, 
                                       stats.norm.cdf(thetas[guessMatrix[1][0]], mus[1], sigmas[1]) - stats.norm.cdf(thetas[guessMatrix[1][0]-1], mus[1], sigmas[1])), 
                                max(0, 
                                    stats.norm.cdf(thetas[guessMatrix[1][1]], mus[1], sigmas[1]) - stats.norm.cdf(thetas[guessMatrix[1][1]-1], mus[1], sigmas[1]))]))
         
        tempProbsStar = [self.gauss(x, i+1.5, sigma_0) for (i, x) in enumerate(thetasStar[1:-1])]

        real_LP_4 = np.sum(np.log(tempProbsStar))
        
        
        real_A1 = real_LP_1 + real_LP_2
        real_A2 = real_LP_3 + real_LP_4
        real_A3 = np.log(self.gauss_trunc(theta, theta_star, sigma_prop, lower_0, upper_0)) - np.log(self.gauss_trunc(theta_star, theta, sigma_prop, lower_0, upper_0))
        comb = np.min([1, np.exp(real_A1 - real_A2 + real_A3)])
        self.assertAlmostEqual(returned_A, comb)
        
        # in this case theta proposal is out of order
        testValues2 = [2.5, 1.3, [-np.inf, 1.5, 2.5, 3.5, np.inf], [[2, 1], [2, 3]], [1.0, 2.0], [1.0, 1.0], 2, 0.5, 1.0, 1.0, 0, 3]
        theta, theta_star, thetas, guessMatrix, mus, sigmas, center, shift, sigma_0, sigma_prop, lower_0, upper_0 = testValues2.copy()
        returned_A2 = theta_accept(*testValues2, printing=True, debug=True)
        thetasStar2 = thetas.copy()
        thetasStar2[center] = theta_star
        
        real_LP_1 = np.sum(np.log([max(0, 
                                       stats.norm.cdf(thetasStar2[guessMatrix[0][0]], mus[0], sigmas[0]) - stats.norm.cdf(thetasStar2[guessMatrix[0][0]-1], mus[0], sigmas[0])), 
                                max(0, 
                                    stats.norm.cdf(thetasStar2[guessMatrix[0][1]], mus[0], sigmas[0]) - stats.norm.cdf(thetasStar2[guessMatrix[0][1]-1], mus[0], sigmas[0])),
                                   max(0, 
                                       stats.norm.cdf(thetasStar2[guessMatrix[1][0]], mus[1], sigmas[1]) - stats.norm.cdf(thetasStar2[guessMatrix[1][0]-1], mus[1], sigmas[1])), 
                                max(0, 
                                    stats.norm.cdf(thetasStar2[guessMatrix[1][1]], mus[1], sigmas[1]) - stats.norm.cdf(thetasStar2[guessMatrix[1][1]-1], mus[1], sigmas[1]))]))
        
        
        tempProbsStar = [self.gauss(x, i+1.5, sigma_0) for (i, x) in enumerate(thetasStar2[1:-1])]
        real_LP_2 = np.sum(np.log(tempProbsStar))

        
        real_LP_3 = np.sum(np.log([max(0, 
                                       stats.norm.cdf(thetas[guessMatrix[0][0]], mus[0], sigmas[0]) - stats.norm.cdf(thetas[guessMatrix[0][0]-1], mus[0], sigmas[0])), 
                                max(0, 
                                    stats.norm.cdf(thetas[guessMatrix[0][1]], mus[0], sigmas[0]) - stats.norm.cdf(thetas[guessMatrix[0][1]-1], mus[0], sigmas[0])),
                                   max(0, 
                                       stats.norm.cdf(thetas[guessMatrix[1][0]], mus[1], sigmas[1]) - stats.norm.cdf(thetas[guessMatrix[1][0]-1], mus[1], sigmas[1])), 
                                max(0, 
                                    stats.norm.cdf(thetas[guessMatrix[1][1]], mus[1], sigmas[1]) - stats.norm.cdf(thetas[guessMatrix[1][1]-1], mus[1], sigmas[1]))]))
         
        tempProbs = [self.gauss(x, i+1.5, sigma_0) for (i, x) in enumerate(thetas[1:-1])]

        real_LP_4 = np.sum(np.log(tempProbs))
        
        
        real_A1 = real_LP_1 + real_LP_2
        real_A2 = real_LP_3 + real_LP_4
        real_A3 = np.log(self.gauss_trunc(theta, theta_star, sigma_prop, lower_0, upper_0)) - np.log(self.gauss_trunc(theta_star, theta, sigma_prop, lower_0, upper_0))
        comb2 = np.min([1, np.exp(real_A1 - real_A2 + real_A3)])
        self.assertAlmostEqual(returned_A2, comb2)
    
runner = unittest.TextTestRunner(failfast=True)
runner.run(initialize_suite(LikelihoodFunctionsTestSuite))  

../var/folders/z3/65sxpv5n3csdhfzm0ysx30940000gp/T/ipykernel_61957/2807394351.py:49: RuntimeWarning: divide by zero encountered in log
  LLQ += np.log(LOP(y, mu, sigma, thetas))
/var/folders/z3/65sxpv5n3csdhfzm0ysx30940000gp/T/ipykernel_61957/2811977516.py:207: RuntimeWarning: divide by zero encountered in log
  real_LP_1 = np.sum(np.log([max(0,
.

in theta prior

 theta has value 1.5

 mu_0 has value 1.5

 1.0 has value sigma_0
in theta prior

 theta has value 2.6

 mu_0 has value 2.5

 1.0 has value sigma_0
in theta prior

 theta has value 3.5

 mu_0 has value 3.5

 1.0 has value sigma_0

 sigma_0 has value 1.0

 ks plus shift has value [1.5, 2.5, 3.5]

 thetas has value [-inf, 1.5, 2.6, 3.5, inf]

 thetaPriors has value [0.3989422804014327, 0.3969525474770118, 0.3989422804014327]

 theta log prior internal has value -2.761815599614018

 thetas has value [-inf, 1.5, 2.6, 3.5, inf]

 shift has value 0.5

 sigma_0 has value 1.0

 LP has value -2.761815599614018
in theta prior

 theta has value 1.5

 mu_0 has value 1.5

 1.0 has value sigma_0
in theta prior

 theta has value 2.5

 mu_0 has value 2.5

 1.0 has value sigma_0
in theta prior

 theta has value 3.5

 mu_0 has value 3.5

 1.0 has value sigma_0

 sigma_0 has value 1.0

 ks plus shift has value [1.5, 2.5, 3.5]

 thetas has value [-inf, 1.5, 2.5, 3.5, inf]

 thetaPriors has


----------------------------------------------------------------------
Ran 3 tests in 0.021s

OK


<unittest.runner.TextTestResult run=3 errors=0 failures=0>